In [ ]:
import json
import os
import sys
import fnmatch


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('BBGM_League_2_all_seasons_Average_Stats.csv')

In [ ]:
y = df.iloc[:,-15:]
X = df.iloc[:,11:-17]
y = y[(X['AST%'] >0) & (X['AST%'] < 100)]
X = X[(X['AST%'] >0) & (X['AST%'] < 100)]
X['MP'] = df.MP
df.columns,X.shape

In [ ]:
for name in ['FG','FGA','3P',"3PA",'FT','FTA',\
             'ORB','DRB','TRB','AST','TOV','STL',"Blk",\
             'PF',"PTS",'OWS','DWS']:
    X[name + 'p36'] = 36* df[name]/np.maximum(1,df.MP)

X = X[[_ for _ in X.columns if '%' in _ or 'r' in _ or 'FGP' in _ or "+/-" in _ or 'p36' in _ or _ in ['PER','MP']]]
X.columns

In [ ]:
from sklearn import neural_network
from sklearn import linear_model
from sklearn import preprocessing


In [ ]:
fexp = preprocessing.PolynomialFeatures(degree=1,interaction_only=True)
scalerX = preprocessing.StandardScaler(with_mean=True,with_std=True)
scalery = preprocessing.StandardScaler(with_mean=True,with_std=True)
prescale_X = scalerX.fit_transform(X)
prescale_y = scalery.fit_transform(y)
prescale_X = fexp.fit_transform(prescale_X)

In [ ]:
clf = neural_network.MLPRegressor((10,5,10),'tanh',solver='adam',max_iter=1000)
#clf = neural_network.MLPRegressor((),'identity',solver='lbfgs')

clf.fit(prescale_X,prescale_y)

In [ ]:
yt = scalery.inverse_transform(clf.predict(prescale_X))
print(np.linalg.norm(yt-y))

In [ ]:
col_names = X.columns
col_names = fexp.get_feature_names(X.columns)

for i,c in enumerate(df.columns[-15:]):
    coeffs = clf.coefs_[-1][:,i]
    v = np.argsort(coeffs)[::-1]
    print(c)
    for i2 in v:
        if abs(coeffs[i2]) < 0.1:
            continue
        print('\t{:25s}\t{:.2f}'.format(col_names[i2],coeffs[i2]))


In [ ]:
tables =np.load('tables_2019.pkl')
teams = tables.keys()

In [ ]:
player_stats = {}
table_columns = {}
for team in tables:
    team_tables =  tables[team]
    for table_name in team_tables:
        if table_name in ['draft-rights','team_and_opponent']:
            continue
        table = team_tables[table_name]
        #print(table_name)
        #print(table.index)
        for row in table.itertuples():
            name = row[0]
            if name == 'Team Totals':
                continue
            player_table = player_stats.get(name,{})
            player_row = player_table.get(table_name,[])
            player_row = player_row + [row]
            
            player_table[table_name] = player_row
            player_stats[name] = player_table
        table_columns[table_name] = table.columns


In [ ]:
table_mask = {}
for table in table_columns:
    table_mask[table] = [_.strip() !=''  for _ in table_columns[table] ]
    table_columns[table] = [_ for _ in table_columns[table] if _.strip() != '']
for player in player_stats:
    for table_in in player_stats[player]:
        if 'on_off' in table_in or 'salaries' in table_in:
            continue
        if len(player_stats[player][table_in]) > 1:
            pass
            #print(player,table_in,'MP' in player_stats[player][table_in][0]._fields)
            #print(player_stats[player][table_in][0])

In [ ]:
for player in player_stats:
    for tt in player_stats[player]:
        new_rows = []
        for tablet in player_stats[player][tt]:
            vector = [_ if _ != '' else '0.5' for _ in tablet[1:]]
            vector = [(float(_.replace('%',''))/100 if type(_) == str and'%' in _ else _) for _ in vector]
            if 'on_off' in tt:
                vector = vector[1:]
            if 'contracts' in tt:
                vector = vector[1:-2]
            if tt in ['salaries2','contracts']:
                vector = [_.replace(',','').replace('$','') for _ in vector]
            try:
                v2 = np.array(vector).astype(np.float)
            except:
                v2 = vector
            new_rows.append(vector)
        a = np.array(new_rows)
        
        if 'MP' in table_columns[tt] and not tt in ['pbp','on_off','on_off_p']:
            a = a.astype(np.float)
            try:
                mins = a[:,table_columns[tt].index('MP')].reshape((-1,1))
                new_rows = ((a.T @ mins)/mins.sum()).T
                a = new_rows
            except:
                print(tt,a.shape,player,a,mins)
                raise

        player_stats[player][tt] = a


In [ ]:
player_vectors = []
player_names = []
for name in player_stats:
    #name = 'Jimmy Butler'
    #name = 'Ben Simmons'
    if not 'totals' in player_stats[name]:
        continue
    player_names.append(name)
    #for stat in X.columns:
    d=    {'FG%':player_stats[name]['totals'][0][6]*100,
        '3P%':player_stats[name]['totals'][0][9]*100,
        'MP':player_stats[name]['per_game'][0][3],
        'FT%':player_stats[name]['totals'][0][16]*100,
        'AtRimFGP':player_stats[name]['shooting'][0][11]*100,
        'LowPostFGP':player_stats[name]['shooting'][0][12]*100,
        'MidRangeFGP':player_stats[name]['shooting'][0][13]*100,
        'TS%':player_stats[name]['advanced'][0][4]*100,
        '3PAr':player_stats[name]['advanced'][0][5]*100,
        'FTr':player_stats[name]['advanced'][0][6]*100,
        'ORB%':player_stats[name]['advanced'][0][7],
        'DRB%':player_stats[name]['advanced'][0][8],
        'TRB%':player_stats[name]['advanced'][0][9],
        'AST%':player_stats[name]['advanced'][0][10],
        'STL%':player_stats[name]['advanced'][0][11],
        'BLK%':player_stats[name]['advanced'][0][12],
        'TOV%':player_stats[name]['advanced'][0][13],
        'USG%':player_stats[name]['advanced'][0][14],
        '+/-':player_stats[name]['advanced'][0][22],
        'FGp36':player_stats[name]['per_minute'][0][4],
        'FGAp36':player_stats[name]['per_minute'][0][5],
        '3Pp36':player_stats[name]['per_minute'][0][6],
        '3PAp36':player_stats[name]['per_minute'][0][7],
        'FTp36':player_stats[name]['per_minute'][0][12],
        'FTAp36':player_stats[name]['per_minute'][0][13],
        'ORBp36':player_stats[name]['per_minute'][0][16],
        'DRBp36':player_stats[name]['per_minute'][0][17],
        'TRBp36':player_stats[name]['per_minute'][0][18],
        'ASTp36':player_stats[name]['per_minute'][0][19],
        'TOVp36':player_stats[name]['per_minute'][0][22],
        'STLp36':player_stats[name]['per_minute'][0][20],
        'Blkp36':player_stats[name]['per_minute'][0][21],
        'PFp36':player_stats[name]['per_minute'][0][23],
        'PTSp36':player_stats[name]['per_minute'][0][24],
        'PER':player_stats[name]['advanced'][0][3],
        'OWSp36':36*player_stats[name]['advanced'][0][16]/player_stats[name]['advanced'][0][2],
        'DWSp36':36*player_stats[name]['advanced'][0][17]/player_stats[name]['advanced'][0][2]}
    player_vectors.append([d[stat] for stat in X.columns])
    if name == 'Ben Simmons':
        print(player_vectors[-1])
X.columns

In [ ]:
#table_columns['per_minute']#,table_columns['advanced'].index('0-3')
#for i,t in enumerate(player_stats[name]['advanced'][0]):
#    print(i,t)


In [ ]:
Xn = np.array(player_vectors)
scalerX2 = preprocessing.StandardScaler(with_mean=True,with_std=True)
ratings = scalery.inverse_transform(clf.predict(fexp.transform(scalerX2.fit_transform(Xn))))
#ratings = scalery.inverse_transform(clf.predict(fexp.transform(scalerX.transform(Xn))))

In [ ]:
player_vectors[player_names.index('Joel Embiid')][13],X.columns[13]

In [ ]:
X.mean(0)

In [ ]:
X.mean(0)-Xn.mean(0)

In [ ]:
for n,v in zip(X.columns,Xn.min(0)):
    print(n,v)

In [ ]:
rosters = {}

for team in tables:
    for name in tables[team]['roster'].index:
        rosters[name] = team
    #print(team,)

In [ ]:
def load_roster(filename):
    base = json.load(open(filename,'rb'))

    
    for i in range(len(base['players'])):
        player = base['players'][i]
        if 'name' in player and player['name'] !='':
            sname = base['players'][i]['name'].split()
            base['players'][i]['firstName'] = sname[0]
            base['players'][i]['lastName'] = ' '.join(sname[1:])
            del base['players'][i]['name']
    players = base['players']
    base['version'] = 32

    return players,base
players,base = load_roster('normed.json')
abbrev_to_tid = {_['abbrev']:_['tid']for _ in base['teams']}
abbrev_to_tid['PHO'] = abbrev_to_tid['PHX']
abbrev_to_tid['CHO'] = abbrev_to_tid['CHA']
abbrev_to_tid['BRK'] = abbrev_to_tid['BKN']
players_discard = [_ for _ in players if  _['tid'] > -3 ]
players = [_ for _ in players if _['tid'] in set([-1,-2,-4,-5])]
y_keys = [_.lower() for _ in y.columns]

y_map = { 'hgt': 'hgt',
   'stre': 'str',
   'spd': 'spd',
   'jmp': 'jmp',
   'endu': 'end',
   'ins': 'ins',
   'dnk': 'dnk',
   'ft': 'ft.1',
   'fg': '2pt',
   'tp': '3pt',
   'diq': 'diq',
   'oiq': 'oiq',
   'drb': 'drb',
   'pss': 'pss',
   'reb': 'reb' }

In [ ]:
import pprint
pp = pprint.PrettyPrinter()
for i,name in enumerate(player_names):
    name = player_names[i]
    sname = name.split(' ')
    new_player = {}
    new_player['firstName'] = sname[0]
    new_player['lastName'] = ' '.join(sname[1:])

    new_player['tid'] = abbrev_to_tid[rosters[name]] if name in rosters else -1
    if name in rosters:
        new_player['pos'] = player_stats[name]['roster'][0][0]
        new_player['weight'] = int(player_stats[name]['roster'][0][2])
        ht = [int(_) for _ in player_stats[name]['roster'][0][1].split('-')]
        new_player['height'] = ht[0]*12 + ht[1]
        new_player['born'] = {'year':int(player_stats[name]['roster'][0][3].split(',')[-1]),'loc':''}
        if 'contracts' in player_stats[name]:
            ctrct = [float(_) for _ in player_stats[name]['contracts'][0] if _ != '0.5']
            if len(ctrct) > 0:
                new_player['contract'] = {'amount':int(round(sum(ctrct)/(1000*len(ctrct)))),'exp':2019+len(ctrct)}
    else:
        continue

    new_player['ratings'] = [{k: int(np.clip(ratings[i,y_keys.index(km)],0,100)) for k,km in y_map.items()}]
    
    players.append(new_player)
    if name in ['LeBron James', "Trevon Duval"]:
        pp.pprint(new_player)

In [ ]:
if False:
    ri = 0
    _ = plt.hist(yt[:,ri],100,normed=True,alpha=0.5,label='predicted')
    _ = plt.hist(y.iloc[:,ri],100,normed=True,alpha=0.5,label='labels')
    _ = plt.hist(ratings[:,ri],100,normed=True,alpha=0.5,label='dataset')
    plt.xlim(-100,100)
    plt.legend()
elif False:
    plt.figure(figsize=(20,20))

    for ri in range(len(X.columns)):
        #ri = 13
        plt.figure()
        #plt.subplot(6,6,ri+1)

        _ = plt.hist(Xn[:,ri],100,density=True,alpha=0.5,label='mine')
        _ = plt.hist(X.iloc[:,ri],100,density=True,alpha=0.5,label='expected')
        plt.legend()
        plt.title(X.columns[ri] + ' ' + str(ri))
        plt.tight_layout()
        plt.xlim(-100,100)


In [ ]:
#Xn[:,1]

In [ ]:
players_discard[0]

In [ ]:
base['players'] = players
with open('auto_roster.json','wt') as fp:
    json.dump(base,fp, indent=4, sort_keys=True)